# Solvent Accessibility Analysis Toolkit - Detailed Tutorial

This tutorial will explain step-by-step the core concepts, code implementation, performance optimization, and parallelization design of the solvent accessibility analysis toolkit.

## Table of Contents

1. [Project Introduction](#1-project-introduction)
2. [Environment Setup](#2-environment-setup)
3. [Core Concepts](#3-core-concepts)
4. [Data Models](#4-data-models)
5. [Algorithm Implementation](#5-algorithm-implementation)
6. [Performance Optimization](#6-performance-optimization)
7. [Parallelization Design](#7-parallelization-design)
8. [Benchmark Testing](#8-benchmark-testing)
9. [Extension Development](#9-extension-development)

## 1. Project Introduction

Protein folding is driven by the hydrophobic effect, which buries hydrophobic residues in the protein core and exposes hydrophilic residues to the aqueous environment. Accurately identifying solvent-accessible residues is crucial for predicting functional regions such as protein-protein interactions and antigen epitopes.

Traditional methods are based on geometric surface area calculations (e.g., FreeSASA). This method directly evaluates the proximity of residues to explicit water molecules, providing a more physically realistic solvent accessibility measure.

**Core Challenge**: Efficiently handling large-scale atom-pair distance calculations in explicit solvation systems (time complexity $O(N×M)$).

**Solution**:
- **Centroid Method**: Calculate the distance from residue centroid to the nearest water molecule
- **Per-Atom Method**: Calculate the proportion of atoms in a residue that can contact water molecules (distance < threshold)
- **Performance Optimization**: Chunked computation, KDTree spatial indexing, vectorized operations, parallel computing

## 2. Environment Setup

Run `setup.sh` to install dependencies. Import and verify required dependencies.

In [ ]:
import sys
import numpy as np
import scipy
import Bio
import freesasa

print(f"Python version: {sys.version}")
print(f"NumPy version: {np.__version__}")
print(f"SciPy version: {scipy.__version__}")
print(f"BioPython version: {Bio.__version__}")
print(
    f"FreeSASA version: {freesasa.__version__ if hasattr(freesasa, '__version__') else '2.2.1 (installed via pip)'}"
)

## 3. Core Concepts

### 3.1 Solvent Accessibility Definition

In protein structures, the solvent accessibility of each amino acid residue describes the degree of contact between the residue and water molecules:

- **Accessible Residue**: Residue surface is fully exposed to the aqueous environment
- **Buried Residue**: Residue surface is surrounded by other residues, isolated from water

### 3.2 Calculation Method Comparison

| Method | Principle | Advantages | Disadvantages |
|--------|-----------|------------|---------------|
| **Geometric Method** (FreeSASA) | Calculate solvent-accessible surface area of atomic spheres | Standard method, comparable results | Ignores explicit distribution of water molecules |
| **Centroid Method** (this tool) | Calculate distance from residue centroid to nearest water molecule | Simple calculation, fast | Ignores residue shape |
| **Per-Atom Method** (this tool) | Calculate proportion of atoms in residue that can contact water molecules | Considers atomic-level details, more accurate | Higher computational cost |

> Note: Our method may be more accurate for instantaneous solvent accessibility. For overall accuracy, dynamic analysis may be required, i.e., processing many trajectory frames xxx_water.pdb and performing statistical analysis.

## 4. Data Models

The project uses Python `dataclass` to define core data structures:

In [ ]:
# Import project modules
import sys

sys.path.append(".")

from core.data_models import (
    ResidueInfo,
    WaterInfo,
    AnalysisConfig,
    MethodType,
)

# Create data model instances
residue = ResidueInfo(
    chain="A", resnum=1, resname="ALA", coord=np.array([1.0, 2.0, 3.0])
)

water_coords = np.array([[1.5, 2.5, 3.5], [2.0, 3.0, 4.0]])
waters = WaterInfo(water_coords)

config = AnalysisConfig(
    threshold=3.5, radius=5.0, fraction_threshold=0.20, chunk_size=5000, num_processes=1
)

print(f"Residue info: {residue}")
print(f"Water molecule count: {len(waters.coords)}")
print(f"Configuration parameters: {config}")

## 5. Algorithm Implementation

### 5.1 Distance Calculator Interface

The project uses strategy pattern to define the distance calculator abstract base class:

In [ ]:
from core.distance_calculator import ChunkedDistanceCalculator

# Create distance calculator instance
calculator = ChunkedDistanceCalculator(chunk_size=5000, num_processes=1)

# Simulate test data
test_residues = [
    ResidueInfo("A", 1, "ALA", np.array([1.0, 2.0, 3.0])),
    ResidueInfo("A", 2, "GLY", np.array([4.0, 5.0, 6.0])),
    ResidueInfo("A", 3, "LEU", np.array([7.0, 8.0, 9.0])),
]

test_waters = WaterInfo(
    np.array([[1.5, 2.5, 3.5], [4.5, 5.5, 6.5], [7.5, 8.5, 9.5], [2.0, 3.0, 4.0]])
)

# Calculate minimum distances
min_distances = calculator.compute_min_distances(test_residues, test_waters)
print(f"Minimum distances: {min_distances}")

# Count water molecules within radius
counts = calculator.count_waters_within_radius(test_residues, test_waters, radius=3.0)
print(f"Water molecules within 3Å radius: {counts}")

### 5.2 Method Factory

Use factory pattern to uniformly create algorithm instances:

In [ ]:
from algorithms.method_factory import MethodFactory

# Create centroid method instance
centroid_config = AnalysisConfig(num_processes=1)
centroid_method = MethodFactory.create_method(MethodType.CENTROID, centroid_config)

# Create per-atom method instance
peratom_config = AnalysisConfig(num_processes=1)
peratom_method = MethodFactory.create_method(MethodType.PERATOM, peratom_config)

print(f"Centroid method type: {centroid_method.get_method_type()}")
print(f"Per-atom method type: {peratom_method.get_method_type()}")

## 6. Performance Optimization

### 6.1 Chunked Computation

When processing large-scale systems, chunked computation can significantly reduce memory peak usage:

In [ ]:
# Compare performance of different chunk sizes
import time


def benchmark_chunk_sizes(residue_count=1000, water_count=10000):
    """Test performance of different chunk sizes"""
    # Generate test data
    residues = [
        ResidueInfo("A", i, "ALA", np.random.randn(3) * 10)
        for i in range(residue_count)
    ]
    waters = WaterInfo(np.random.randn(water_count, 3) * 20)

    chunk_sizes = [100, 500, 1000, 5000, 10000]
    results = {}

    for chunk_size in chunk_sizes:
        calculator = ChunkedDistanceCalculator(chunk_size=chunk_size)
        start_time = time.perf_counter()
        distances = calculator.compute_min_distances(residues, waters)
        end_time = time.perf_counter()
        results[chunk_size] = end_time - start_time

    return results


# Run benchmark test
results = benchmark_chunk_sizes(residue_count=500, water_count=5000)
print("Computation time for different chunk sizes:")
for chunk_size, elapsed in results.items():
    print(f"  chunk_size={chunk_size}: {elapsed:.4f}s")

### 6.2 KDTree Spatial Indexing

Use SciPy's KDTree to accelerate nearest neighbor search:

In [ ]:
from scipy.spatial import KDTree


# Demonstrate KDTree acceleration effect
def compare_search_methods(n_points=1000, n_queries=100):
    """Compare performance of brute-force search and KDTree search"""
    # Generate random points
    points = np.random.randn(n_points, 3) * 10
    query_points = np.random.randn(n_queries, 3) * 10

    # Brute-force search
    start_time = time.perf_counter()
    brute_force_results = []
    for q in query_points:
        distances = np.linalg.norm(points - q, axis=1)
        min_idx = np.argmin(distances)
        brute_force_results.append((min_idx, distances[min_idx]))
    brute_time = time.perf_counter() - start_time

    # KDTree search
    start_time = time.perf_counter()
    tree = KDTree(points)
    distances, indices = tree.query(query_points, k=1)
    kdtree_time = time.perf_counter() - start_time

    return brute_time, kdtree_time


brute_time, kdtree_time = compare_search_methods(n_points=1000, n_queries=100)
print(f"Brute-force search time: {brute_time:.4f}s")
print(f"KDTree search time: {kdtree_time:.4f}s")
print(f"Speedup ratio: {brute_time/kdtree_time:.2f}x")

## 7. Parallelization Design

### 7.1 Parallelization Proposition Verification

**Proposition**: "Since the distance calculation between each amino acid and its nearest water molecule does not depend on previous amino acid calculations, theoretically each amino acid can be computed in parallel after the tree is built."

**Verification**: Proposition holds
- Computational independence: After building KDTree, each residue query is completely independent
- Read-only operation: KDTree query is thread-safe read-only operation
- No state sharing: Conforms to "embarrassingly parallel" problem characteristics

### 7.2 Parallelization Components

In [ ]:
from core.parallel import ParallelKDTreeQuery


# Demonstrate parallel query
def demo_parallel_query():
    # Create test data
    n_points = 100
    points = np.random.randn(n_points, 3) * 10
    tree = KDTree(points)

    # Query points
    query_points = np.random.randn(50, 3) * 10
    radius = 2.0

    # Serial query
    start_time = time.perf_counter()
    serial_results = [tree.query_ball_point(p, radius) for p in query_points]
    serial_time = time.perf_counter() - start_time

    # Parallel query (2 threads)
    parallel_query = ParallelKDTreeQuery(tree, num_workers=2)
    start_time = time.perf_counter()
    parallel_results = parallel_query.query_ball_point_parallel(query_points, radius)
    parallel_time = time.perf_counter() - start_time

    # Verify result consistency
    consistent = all(len(s) == len(p) for s, p in zip(serial_results, parallel_results))

    return serial_time, parallel_time, consistent


serial_time, parallel_time, consistent = demo_parallel_query()
print(f"Serial query time: {serial_time:.4f}s")
print(f"Parallel query time: {parallel_time:.4f}s")
print(f"Speedup ratio: {serial_time/parallel_time:.2f}x")
print(f"Result consistency: {'✓' if consistent else '✗'}")

## 8. Benchmark Testing

### 8.1 Complete System Test

Use the PDB files provided by the project for end-to-end testing:

In [ ]:
# Load PDB files
from io_utils.pdb_loader import PDBLoader


def analyze_pdb_file(pdb_path):
    """Analyze a single PDB file"""
    loader = PDBLoader(quiet=True)
    residues, waters, structure = loader.load(pdb_path)

    print(f"File: {pdb_path}")
    print(f"Residue count: {len(residues)}")
    print(
        f"Water molecule count: {len(waters.coords) if waters.coords is not None else 0}"
    )

    # Test both methods
    config = AnalysisConfig(num_processes=1)

    # Centroid method
    centroid_method = MethodFactory.create_method(MethodType.CENTROID, config)
    start_time = time.perf_counter()
    centroid_results = centroid_method.analyze(residues, waters, structure)
    centroid_time = time.perf_counter() - start_time
    centroid_accessible = sum(1 for r in centroid_results if r.accessible)

    # Per-atom method
    peratom_method = MethodFactory.create_method(MethodType.PERATOM, config)
    start_time = time.perf_counter()
    peratom_results = peratom_method.analyze(residues, waters, structure)
    peratom_time = time.perf_counter() - start_time
    peratom_accessible = sum(1 for r in peratom_results if r.accessible)

    print(
        f"Centroid method: {centroid_time:.3f}s, accessible residues: {centroid_accessible}/{len(centroid_results)}"
    )
    print(
        f"Per-atom method: {peratom_time:.3f}s, accessible residues: {peratom_accessible}/{len(peratom_results)}"
    )
    print()

    return {
        "centroid_time": centroid_time,
        "centroid_accessible": centroid_accessible,
        "peratom_time": peratom_time,
        "peratom_accessible": peratom_accessible,
    }


# Test available PDB files
import os

pdb_files = []
if os.path.exists("./pdb"):
    pdb_files = [f for f in os.listdir("./pdb") if f.endswith(".pdb")]
    pdb_files = pdb_files[:3]  # Test first 3 files

results = {}
for pdb_file in pdb_files:
    try:
        file_path = os.path.join("./pdb", pdb_file)
        results[pdb_file] = analyze_pdb_file(file_path)
    except Exception as e:
        print(f"Error analyzing file {pdb_file}: {e}")

### 8.2 Parallelization Performance Comparison

Use the benchmark testing script provided by the project:

In [ ]:
# Run benchmark test script
!python benchmark_parallel.py --wet-pdb ./pdb/SUMO1_water.pdb --dry-pdb ./pdb/SUMO1.pdb --processes 1 2 --runs 2 --methods centroid

## 9. Extension Development

### 9.1 Adding New Distance Algorithms

Implement the `DistanceCalculator` abstract base class:

In [ ]:
from core.distance_calculator import DistanceCalculator


class CustomDistanceCalculator(DistanceCalculator):
    """Custom distance calculator example"""

    def __init__(self, custom_param: float = 1.0):
        self.custom_param = custom_param

    def compute_min_distances(self, residues, waters):
        """Custom minimum distance calculation"""
        if waters.is_empty():
            return np.full(len(residues), np.inf)

        # Example: Use weighted Euclidean distance
        res_coords = np.vstack([r.coord for r in residues])
        water_coords = waters.coords

        # Calculate weighted distances
        n_res = len(residues)
        min_distances = np.full(n_res, np.inf)

        for i, r_coord in enumerate(res_coords):
            distances = np.linalg.norm(water_coords - r_coord, axis=1)
            weighted_distances = distances * self.custom_param  # Apply custom weight
            min_distances[i] = np.min(weighted_distances)

        return min_distances

    def count_waters_within_radius(self, residues, waters, radius):
        """Custom water molecule count within radius"""
        if waters.is_empty():
            return np.zeros(len(residues), dtype=int)

        from scipy.spatial import KDTree

        res_coords = np.vstack([r.coord for r in residues])
        water_tree = KDTree(waters.coords)

        counts = np.zeros(len(residues), dtype=int)
        for i, coord in enumerate(res_coords):
            indices = water_tree.query_ball_point(coord, radius)
            counts[i] = len(indices)

        return counts


# Test custom calculator
custom_calc = CustomDistanceCalculator(custom_param=1.5)
print(f"Custom calculator created successfully: {custom_calc}")

### 9.2 Adding New Evaluation Rules

Implement the `AccessibilityEvaluator` abstract base class:

In [ ]:
from core.accessibility_evaluator import AccessibilityEvaluator
from core.data_models import AccessibilityResult, MethodType


class CustomEvaluator(AccessibilityEvaluator):
    """Custom accessibility evaluator example"""

    def __init__(self, method: MethodType = MethodType.CENTROID):
        """Initialize custom evaluator

        Args:
            method: Evaluator corresponding method type, default is centroid method
        """
        self.method = method

    def evaluate(self, residues, min_distances, water_counts, config):
        """Custom evaluation logic"""
        results = []

        for i, residue in enumerate(residues):
            min_dist = min_distances[i]
            water_count = water_counts[i]

            # Custom rule: Combine distance and water molecule count
            if min_dist < config.threshold and water_count > 2:
                accessible = True
            else:
                accessible = False

            result = AccessibilityResult(
                residue=residue,
                min_distance=min_dist,
                water_count=water_count,
                accessible=accessible,
                method=self.method,  # Use method type set during initialization
            )
            results.append(result)

        return results


# Test custom evaluator
custom_eval = CustomEvaluator(method=MethodType.CENTROID)
print(f"Custom evaluator created successfully: {custom_eval}")
print(f"Evaluator method type: {custom_eval.method}")

## Summary

This tutorial has detailed the solvent accessibility analysis toolkit's:

1. **Core Concepts**: Physical meaning of protein solvent accessibility and calculation methods
2. **Data Models**: Core data structures defined using Python dataclass
3. **Algorithm Implementation**: Strategy pattern design of centroid and per-atom methods
4. **Performance Optimization**: Chunked computation, KDTree indexing, vectorized operations
5. **Parallelization Design**: Parallel components based on Python 3.14 free-threading feature
6. **Benchmark Testing**: Algorithm performance comparison and parallelization effectiveness analysis
7. **Extension Development**: How to add new algorithms and evaluation rules

### Next Steps

- Try analyzing your own PDB files
- Implement custom distance algorithms or evaluation rules
- Optimize performance parameters (chunk_size, num_processes)
- Explore GPU acceleration possibilities (essentially super-parallel, but need to analyze computational precision first)
- If original data precision is determined or excessively high, implement multiplication by a factor during reading to convert float calculations to integer calculations

### Reference Resources

- [BioPython Documentation](https://biopython.org/)
- [FreeSASA Documentation](https://freesasa.github.io/)
- [SciPy Spatial Algorithms](https://docs.scipy.org/doc/scipy/reference/spatial.html)